### Web Scraping 2: DevTools, elementos HTML y scraping con REQUESTS y BEAUTIFULSOUP

In [4]:
from bs4 import BeautifulSoup  # Importar la clase BeautifulSoup para el análisis HTML
import pyarrow as pa  # Importar la biblioteca pyarrow para trabajar con Arrow
import pandas as pd  # Importar la biblioteca pandas para trabajar con DataFrames
import requests  # Importar la biblioteca requests para realizar solicitudes HTTP
import pyarrow.parquet as pq  # Importar la biblioteca pyarrow para trabajar con archivos Parquet

Probando dentro de la WEB para ver las cosas que queremos scrapear

https://nominatim.openstreetmap.org/ui/reverse.html
https://nominatim.openstreetmap.org/ui/reverse.html?lat=-34.6037&lon=-58.3816&zoom=18

In [ ]:
#Entraremos aqui y verificaremo el valor de la clave 'suburb' o 'neighbourhood'
#https://nominatim.openstreetmap.org/reverse?lat=-34.6037&lon=-58.3816&format=json

In [5]:
# Definir una función llamada obtener_barrio que toma las coordenadas de latitud y longitud como entrada
def obtener_barrio(latitud, longitud):
    # Construir la URL para la consulta a la API de OpenStreetMap Nominatim
    url = f'https://nominatim.openstreetmap.org/reverse?lat={latitud}&lon={longitud}&format=json'
    
    # Realizar una solicitud HTTP GET a la URL construida
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Extraer los datos JSON de la respuesta
        data = response.json()
        
        # Verificar si la clave 'address' está presente en los datos JSON
        if 'address' in data:
            # Verificar si la clave 'suburb' está presente en la dirección
            if 'suburb' in data['address']:
                # Devolver el nombre del barrio si está presente en 'suburb'
                return data['address']['suburb']
            # Si 'suburb' no está presente, verificar si 'neighbourhood' está presente en la dirección
            elif 'neighbourhood' in data['address']:
                # Devolver el nombre del barrio si está presente en 'neighbourhood'
                return data['address']['neighbourhood']
    
    # Devolver None si no se puede determinar el barrio
    return None

# Ejemplo de uso de la función obtener_barrio
latitud = -34.6037  # Definir la latitud de un lugar en Buenos Aires
longitud = -58.3816  # Definir la longitud de un lugar en Buenos Aires

# Llamar a la función obtener_barrio con las coordenadas especificadas
barrio = obtener_barrio(latitud, longitud)

# Verificar si se pudo determinar el barrio para las coordenadas proporcionadas
if barrio:
    # Imprimir un mensaje indicando el nombre del barrio si se pudo determinar
    print(f'Las coordenadas ({latitud}, {longitud}) corresponden al barrio de {barrio} en Buenos Aires.')
else:
    # Imprimir un mensaje indicando que no se pudo determinar el barrio
    print('No se pudo determinar el barrio para las coordenadas proporcionadas.')



Las coordenadas (-34.6037, -58.3816) corresponden al barrio de San Nicolás en Buenos Aires.


ahora para nuestro df de coordenadas

In [6]:
import pandas as pd  # Importar la biblioteca pandas para trabajar con DataFrames
import requests  # Importar la biblioteca requests para realizar solicitudes HTTP
import pyarrow.parquet as pq  # Importar la biblioteca pyarrow para trabajar con archivos Parquet

# Definir una función llamada obtener_barrio que toma las coordenadas de latitud y longitud como entrada
def obtener_barrio(latitud, longitud):
    # Construir la URL para la consulta a la API de OpenStreetMap Nominatim
    url = f'https://nominatim.openstreetmap.org/reverse?lat={latitud}&lon={longitud}&format=json'
    
    # Realizar una solicitud HTTP GET a la URL construida
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Extraer los datos JSON de la respuesta
        data = response.json()
        
        # Verificar si la clave 'address' está presente en los datos JSON
        if 'address' in data:
            # Verificar si la clave 'suburb' está presente en la dirección
            if 'suburb' in data['address']:
                # Devolver el nombre del barrio si está presente en 'suburb'
                return data['address']['suburb']
            # Si 'suburb' no está presente, verificar si 'neighbourhood' está presente en la dirección
            elif 'neighbourhood' in data['address']:
                # Devolver el nombre del barrio si está presente en 'neighbourhood'
                return data['address']['neighbourhood']
    
    # Devolver None si no se puede determinar el barrio
    return None

# Definir una función llamada obtener_barrios_para_coordenadas que toma un DataFrame como entrada
def obtener_barrios_para_coordenadas(df):
    barrios = []  # Crear una lista vacía para almacenar los nombres de los barrios
    for index, row in df.iterrows():  # Iterar sobre cada fila del DataFrame
        barrio = obtener_barrio(row['pos y'], row['pos x'])  # Obtener el nombre del barrio para las coordenadas de la fila
        barrios.append(barrio)  # Agregar el nombre del barrio a la lista de barrios
    return barrios  # Devolver la lista de barrios

# Leer el archivo reviews.parquet en un DataFrame
df = pq.read_table("../0-DATA/homicidios.parquet").to_pandas()

# Seleccionar las columnas de coordenadas 'pos x' y 'pos y' del DataFrame
df = df[["pos x","pos y"]]

# Obtener el nombre del barrio para cada fila del DataFrame
barrios = obtener_barrios_para_coordenadas(df)

# Agregar los nombres de los barrios como una nueva columna llamada 'Barrio' al DataFrame
df['Barrio'] = barrios

# Mostrar el DataFrame con los nombres de los barrios
print(df)



            pos x         pos y             Barrio
0    -58.47533969  -34.68757022    Villa Riachuelo
1    -58.50877521  -34.66977709          Mataderos
2    -58.39040293  -34.63189362       Constitución
3    -58.46503904  -34.68092974       Villa Lugano
4    -58.38718297  -34.62246630       Constitución
..            ...           ...                ...
712  -58.43353773  -34.64561636      Nueva Pompeya
713  -58.46739825  -34.65117757  Parque Avellaneda
714  -58.47293407  -34.61984745             Flores
715  -58.47066794  -34.65021673  Parque Avellaneda
716  -58.37976155  -34.58679619             Retiro

[717 rows x 3 columns]


In [7]:
df

,pos x,pos y,Barrio
0,-58.47533969,-34.68757022,Villa Riachuelo
1,-58.50877521,-34.66977709,Mataderos
2,-58.39040293,-34.63189362,Constitución
3,-58.46503904,-34.68092974,Villa Lugano
4,-58.38718297,-34.62246630,Constitución
...,...,...,...
712,-58.43353773,-34.64561636,Nueva Pompeya
713,-58.46739825,-34.65117757,Parque Avellaneda
714,-58.47293407,-34.61984745,Flores
715,-58.47066794,-34.65021673,Parque Avellaneda


In [8]:
#guardamos en un CSV
df.to_csv("../0-DATA/BarriosBA.csv", index=False, encoding="utf-8") 

In [9]:
#Procedemos de la misma manera guardando en un archivo .parquet
itemsl = pd.read_csv("../0-DATA/BarriosBA.csv") 
tabla = pa.Table.from_pandas(itemsl) 
pq.write_table(tabla,"../0-DATA/BarriosBA.parquet") 